In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as mpl
import pymc as pm
import arviz as az
import pandas as pd
import os
from datetime import datetime
import sys
from math import atan,cos,sin
from vd_class import vd_8dof
from point import Point

### Set up vehicle model for steering test

In [ ]:
# A ramp steer
pt1 = Point(0,0)
pt2 = Point(3.7,0.2)
ramp_st_3 = pt1.get_eq(pt2)

# Used for obtaining the state from the chrono vehicle
n1  = 700
n2 = 1070


# The time duration of the simulation
st_time = 0.
end_time = 3.7


# The times for which we want the simulation
time  = np.arange(st_time,end_time,0.01)
t_eval = np.arange(st_time,end_time,0.01)

def zero_throt(t):
    return 0 * t
def brake_tor(t):
    return 0 * t

state = pd.read_csv("data/simp_ramp.csv",sep=',',header='infer')

st = {'x' : state['x'][n1]-state['x'][n1],'y':state['y'][n1]-state['y'][n1],'u':state['vx'][n1],
'v':state['vy'][n1],'psi':state['yaw'][n1],'phi':state['roll'][n1],'wx':state['roll_rate'][n1],
'wz':state['yaw_rate'][n1],'wlf' : state['wlf'][n1],'wlr' : state['wlr'][n1],'wrf' : state['wrf'][n1],'wrr' : state['wrr'][n1]}

vehicle = vd_8dof(states = st)

# Set the steering and the throttle functions we just created above
vehicle.set_steering(ramp_st_3)
vehicle.set_throttle(zero_throt,gr=0.3*0.2)
vehicle.set_braking(brake_tor)
vehicle.debug = 0

### Get the chain and load the files

In [ ]:
# The chain file
filename = "20220611_230716"
idata = az.from_netcdf('./results/' + filename + ".nc")
dataFileName = "data/vd_chrono_ramp_1.npy"
##The data in npy format
with open(dataFileName, 'rb') as f:
    data = np.load(f)


In [ ]:
save = False

### Set up the posteriors and the priors on parameters of interest

In [ ]:
shape = idata['posterior']['Cf'].shape

posterior_Cf = np.asarray(idata['posterior']['Cf']).reshape(shape[0]*shape[1],)
posterior_Cr = np.asarray(idata['posterior']['Cr']).reshape(shape[0]*shape[1],)
posterior_krof = np.asarray(idata['posterior']['krof']).reshape(shape[0]*shape[1],)
posterior_kror =np.asarray(idata['posterior']['kror']).reshape(shape[0]*shape[1],)
posterior_brof = np.asarray(idata['posterior']['brof']).reshape(shape[0]*shape[1],)
posterior_bror =np.asarray(idata['posterior']['bror']).reshape(shape[0]*shape[1],)


prior_Cf = np.random.uniform(low=20000,high=80000,size=(shape[0]*shape[1]),)
prior_Cr = np.random.uniform(low=20000,high=80000,size=(shape[0]*shape[1]),)
prior_krof = np.random.uniform(low=1000,high=50000,size=(shape[0]*shape[1]),)
prior_kror = np.random.uniform(low=1000,high=50000,size=(shape[0]*shape[1]),)
prior_bro = np.random.uniform(low=200,high=60000,size=(shape[0]*shape[1]),)
prior_brof = prior_bro/2
prior_bror = prior_bro/2

### Sample from the posterior

In [ ]:
#Number of posterior samples
no_post = 100


random_points = np.random.randint(low=0,high=shape[0]*shape[1],size=(no_post,),dtype=int)

#The posterior lines
posterior_LV = np.empty((no_post,time.shape[0]))
posterior_YW = np.empty((no_post,time.shape[0]))
posterior_RA = np.empty((no_post,time.shape[0]))
posterior_RR = np.empty((no_post,time.shape[0]))

for i in range(0,no_post):
    vehicle.update_params(Cf=posterior_Cf[random_points[i]],
                      Cr = posterior_Cr[random_points[i]],krof = posterior_krof[random_points[i]],kror = posterior_kror[random_points[i]],
                      brof = posterior_brof[random_points[i]],bror = posterior_bror[random_points[i]],
                         m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                          Jx = 1289,Jz = 4519,Jxz = 3.265,r0=0.47,ktf=326332,ktr=326332,hrcf=0.379,hrcr=0.327,Jw=11,
                          Cxf = 10300,Cxr = 10300,rr=0.0125)
    mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 1e-2)
    mod = np.transpose(mod)
    vehicle.reset_state(init_state=st)

    posterior_RA[i] = mod[5,:]
    posterior_RR[i] = mod[6,:]
    posterior_LV[i] = mod[3,:]
    posterior_YW[i] = mod[7,:]


### Sample from the prior

In [ ]:
#Number of prior samples
no_prior = 100


random_points = np.random.randint(low=0,high=shape[0]*shape[1],size=(no_post,),dtype=int)

#The prior lines
prior_LV = np.empty((no_post,time.shape[0]))
prior_YW = np.empty((no_post,time.shape[0]))
prior_RA = np.empty((no_post,time.shape[0]))
prior_RR = np.empty((no_post,time.shape[0]))

for i in range(0,no_prior):
    vehicle.update_params(Cf=prior_Cf[random_points[i]],
                      Cr = prior_Cr[random_points[i]],krof = prior_krof[random_points[i]],kror = prior_kror[random_points[i]],
                      brof = prior_brof[random_points[i]],bror = prior_bror[random_points[i]],
                         m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                          Jx = 1289,Jz = 4519,Jxz = 3.265,r0=0.47,ktf=326332,ktr=326332,hrcf=0.379,hrcr=0.327,Jw=11,
                          Cxf = 10300,Cxr = 10300,rr=0.0125)
    mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 1e-2)
    mod = np.transpose(mod)
    vehicle.reset_state(init_state=st)

    prior_RA[i] = mod[5,:]
    prior_RR[i] = mod[6,:]
    prior_LV[i] = mod[3,:]
    prior_YW[i] = mod[7,:]


### Expectations

In [ ]:
post_exp_LV = posterior_LV.mean(axis=0)
post_exp_YW = posterior_YW.mean(axis=0)
post_exp_RA = posterior_RA.mean(axis=0)
post_exp_RR = posterior_RR.mean(axis=0)

prior_exp_LV = prior_LV.mean(axis=0)
prior_exp_YW = prior_YW.mean(axis=0)
prior_exp_RA = prior_RA.mean(axis=0)
prior_exp_RR = prior_RR.mean(axis=0)

### Mean posterior

In [ ]:
## To plot the mean posterior 
post_Cf = idata['posterior']['Cf'].mean()
post_Cr = idata['posterior']['Cr'].mean()
post_krof = idata['posterior']['krof'].mean()
post_kror = idata['posterior']['kror'].mean()
post_brof = idata['posterior']['brof'].mean()
post_bror = idata['posterior']['bror'].mean()

vehicle.update_params(Cf = post_Cf,Cr = post_Cr,krof = post_krof,kror = post_kror,brof = post_brof,bror = post_bror,
                         m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                      Jx = 1289,Jz = 4519,Jxz = 3.265,r0=0.47,ktf=326332,ktr=326332,hrcf=0.379,hrcr=0.327,Jw=11,
                      Cxf = 10300,Cxr = 10300,rr=0.0125)
mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 1e-2)
mod = np.transpose(mod)
vehicle.reset_state(init_state=st)

post_mean_RA = mod[5,:].reshape(-1,1)
post_mean_RR = mod[6,:].reshape(-1,1)
post_mean_LV = mod[3,:].reshape(-1,1)
post_mean_YW = mod[7,:].reshape(-1,1)

In [ ]:
## To plot the mean priorerior 
prior_Cf = prior_Cf.mean()
prior_Cr = prior_Cr.mean()
prior_krof = prior_krof.mean()
prior_kror = prior_kror.mean()
prior_brof = prior_brof.mean()
prior_bror = prior_bror.mean()

vehicle.update_params(Cf = prior_Cf,Cr = prior_Cr,krof = prior_krof,kror = prior_kror,brof = prior_brof,bror = prior_bror,
                         m=2097.85,muf=127.866,mur=129.98,a= 1.6889,b =1.6889,h = 0.713,cf = 1.82,cr = 1.82,
                      Jx = 1289,Jz = 4519,Jxz = 3.265,r0=0.47,ktf=326332,ktr=326332,hrcf=0.379,hrcr=0.327,Jw=11,
                      Cxf = 10300,Cxr = 10300,rr=0.0125)
mod = vehicle.solve_half_impl(t_span = [t_eval[0],t_eval[-1]],t_eval = t_eval,tbar = 1e-2)
mod = np.transpose(mod)
vehicle.reset_state(init_state=st)

prior_mean_RA = mod[5,:].reshape(-1,1)
prior_mean_RR = mod[6,:].reshape(-1,1)
prior_mean_LV = mod[3,:].reshape(-1,1)
prior_mean_YW = mod[7,:].reshape(-1,1)

## Priors

### Lateral Velocity Prior

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,prior_LV[i,:],'y',alpha = 0.2)
   
mpl.plot(time,data[0,:],'r',label ='noisy data')
mpl.plot(time,prior_exp_LV,'k--',linewidth=2,label='prior expectation')
mpl.plot(time,prior_mean_LV,'g--',linewidth=2,label='prior mean')
mpl.title("Lateral Velocity - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Lateral Velocity (m/s)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LV_st_prior_1.png',facecolor = 'w')


### Yaw rate

In [ ]:
mpl.figure(figsize = (6,6))
mpl.plot(time,data[3,:],'r',label ='noisy data',alpha=0.75)
for i in range(0,no_post):
    mpl.plot(time,prior_YW[i,:],'y',alpha=0.2)

mpl.plot(time,prior_exp_YW,'k--',linewidth=2,label='prior expectiation')
mpl.title("Yaw Rate - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Yaw rate (rad/s)',fontsize = 12)
mpl.plot(time,prior_mean_YW,'g--',linewidth=2,label='prior mean')
mpl.legend()
if(save):
    mpl.savefig('./images/YR_st_prior_1.png',facecolor = 'w')

### Roll angle

In [ ]:
mpl.figure(figsize = (6,6))
mpl.plot(time,data[1,:],'r',label ='noisy data',alpha=0.75)
for i in range(0,no_post):
    mpl.plot(time,prior_RA[i,:],'y',alpha=0.2)

mpl.plot(time,prior_exp_RA,'k--',linewidth=2,label='prior expectiation')
mpl.title("Roll Angle - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Roll Angle (rad)',fontsize = 12)
mpl.ylim(top=0.4)
mpl.plot(time,prior_mean_RA,'g--',linewidth=2,label='prior mean')
mpl.legend()
if(save):
    mpl.savefig('./images/RA_st_prior_1.png',facecolor = 'w')

### Roll Rate

In [ ]:
mpl.figure(figsize = (6,6))
mpl.plot(time,data[2,:],'r',label ='noisy data',alpha=0.75)
for i in range(0,no_post):
    mpl.plot(time,prior_RR[i,:],'y',alpha=0.2)

mpl.plot(time,prior_exp_RR,'k--',linewidth=2,label='prior expectiation')
mpl.title("Roll Rate - Prior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Roll rate (rad/s)',fontsize = 12)
mpl.ylim(bottom=-0.1,top=0.2)
mpl.plot(time,prior_mean_RR,'g--',linewidth=2,label='prior mean')
mpl.legend()
if(save):
    mpl.savefig('./images/RR_st_prior_1.png',facecolor = 'w')

## Posteriors

### Lateral Velocity

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_LV[i,:],'b',alpha=0.75)
    
mpl.plot(time,data[0,:],'r',label ='noisy data')
mpl.plot(time,post_exp_LV,'k',linewidth = 2,label='posterior expectiation')
mpl.plot(time,post_mean_LV,'g',linewidth=2,label='posterior mean')
mpl.title("Lateral Velocity - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Lateral Velocity (m/s)',fontsize = 12)
mpl.legend()
if(save):
    mpl.savefig('./images/LV_st_post_1.png',facecolor = 'w')


### Yaw rate psoterior

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_YW[i,:],'b',alpha=0.2)
mpl.plot(time,data[3,:],'r',label ='noisy data',alpha=0.75) 
mpl.plot(time,post_exp_YW,'k',linewidth = 2,label='posterior expectation')
mpl.title("Yaw Rate - posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Yaw rate (rad/s)',fontsize = 12)
mpl.plot(time,post_mean_YW,'g',linewidth=2,label='posterior mean')

mpl.legend()
if(save):
    mpl.savefig('./images/YR_st_post_1.png',facecolor = 'w')


### Roll angle prior

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_RA[i,:],'b',alpha=0.2)
    
mpl.plot(time,data[1,:],'r',label ='noisy data',alpha=0.75)
mpl.plot(time,post_exp_RA,'k',linewidth = 2,label='posterior expectation')
mpl.title("Roll Angle - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Roll Angle (rad)',fontsize = 12)
mpl.plot(time,post_mean_RA,'g',linewidth=2,label='posterior mean')
mpl.legend()
if(save):
    mpl.savefig('./images/RA_st_post_1.png',facecolor = 'w')

### Roll rate

In [ ]:
mpl.figure(figsize = (6,6))

for i in range(0,no_post):
    mpl.plot(time,posterior_RR[i,:],'b',alpha=0.2)
    
mpl.plot(time,data[2,:],'r',label ='noisy data',alpha=0.75)
mpl.plot(time,post_exp_RR,'k',linewidth = 2,label='posterior expectation')
mpl.title("Roll Rate - Posterior",fontsize = 20)
mpl.xlabel('Time (S)',fontsize = 12)
mpl.ylabel('Roll rate (rad/s)',fontsize = 12)
mpl.plot(time,post_mean_RR,'g',linewidth=2,label='posterior mean')
mpl.legend()
if(save):
    mpl.savefig('./images/RR_st_post_1.png',facecolor = 'w')

### RMSE

In [ ]:
msae_post_LV = np.mean(np.sqrt((np.sum((posterior_LV - data[0,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_post_LV

In [ ]:
msae_prior_LV = np.mean(np.sqrt((np.sum((prior_LV - data[0,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_prior_LV

In [ ]:
msae_post_RA = np.mean(np.sqrt((np.sum((posterior_RA - data[1,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_post_RA

In [ ]:
msae_prior_RA = np.mean(np.sqrt((np.sum((prior_RA - data[1,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_prior_RA

In [ ]:
msae_post_RR = np.mean(np.sqrt((np.sum((posterior_RR - data[2,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_post_RR

In [ ]:
msae_prior_RR = np.mean(np.sqrt((np.sum((prior_RR - data[2,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_prior_RR

In [ ]:
msae_post_YW = np.mean(np.sqrt((np.sum((posterior_YW - data[3,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_post_YW

In [ ]:
msae_prior_YW = np.mean(np.sqrt((np.sum((prior_YW - data[3,:])**2,axis=1)/data.shape[1])))

In [ ]:
msae_prior_YW